# **딥러닝 따라하기**

<center><img src = "https://github.com/Jangrae/img/blob/master/computing.jpg?raw=true" width=800/></center>

## **1. 환경 준비**

- 이후에 사용할 라이브러리를 모두 불러옵니다.

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings

warnings.filterwarnings(action='ignore')

## **2. 모델링 1: 광고 효과 예측**

- Advertising 데이터로 모델링합니다.

### (1) 데이터 전처리

- 모델링을 위해 필요한 전처리를 진행합니다.

#### 1) 데이터 준비

- 사용할 데이터를 불러옵니다.

In [2]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/advertising.csv'
data = pd.read_csv(path)

# 확인
data.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


**데이터 설명**

- TV: TV 광고비
- Radio: 라디오 광고비
- Newspaper: 신문 광고비
- Sales: 판매 건수

- Target을 확인하고 데이터 셋을 x, y로 분리합니다.

In [3]:
# x, y 분리
target = 'Sales'
x = data.drop(columns=target)
y = data.loc[:, target]

#### 2) 가변수화

- 필요한 경우 가변수화를 수행합니다.

In [4]:
# 없음

#### 3) 데이터 분할

- 데이터 셋을 학습용, 검증용 데이터로 분리합니다.

In [5]:
# 학습용, 검증용 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

### (2) 머신러닝 모델링(선형회귀)

- 딥러닝 모델링을 하기 전에 머신러닝 모델링은 어떻게 했었는지 기억을 되살려봅니다.

In [6]:
# 불러오기
from sklearn.linear_model import LinearRegression

In [ ]:
# 선언하기
model = LinearRegression()

In [8]:
# 학습하기
model.fit(x_train, y_train)

LinearRegression()

In [9]:
# 예측하기
y_pred = model.predict(x_val)

In [10]:
# 평가하기
print('* MAE:', mean_absolute_error(y_val, y_pred))
print('* R2 Score:', r2_score(y_val, y_pred))

* MAE: 1.0402154012924716
* R2 Score: 0.8927605914615384


### (3) 딥러닝 모델링

- 이제 딥러닝 모델링을 다음과 같은 순서로 진행합니다.
    - 전처리
    - 함수 불러오기
    - 모델 선언
    - 모델 학습
    - 예측
    - 성능 평가

#### 1) 스케일링

- 딥러닝 모델링을 위해서는 스케일링이 필수입니다.

In [11]:
data.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [11]:
# 스케일링
scaler = StandardScaler()
scaled = scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled, columns=data.columns)
scaled_df.head()

,TV,Radio,Newspaper,Sales
0,0.969852,0.981522,1.778945,1.552053
1,-1.197376,1.082808,0.669579,-0.696046
2,-1.516155,1.528463,1.783549,-0.907406
3,0.052050,1.217855,1.286405,0.860330
4,0.394182,-0.841614,1.281802,-0.215683


#### 2) 함수 불러오기

- 이후 사용할 함수들을 불러옵니다.

In [16]:
# 함수 불러오기
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### 3) 모델 선언

<img src = "https://github.com/Jangrae/img/blob/master/deep_sample01.png?raw=true" width=500 align="left"/>

In [20]:
# 메모리 정리
tf.keras.backend.clear_session()

# 입력 값 개수
nfeatures = x.shape[1]

# Sequential 모델 선언
model = Sequential([
    Dense(64, activation='relu', input_shape=(nfeatures,)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# 모델 요약
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,369 (9.25 KB)

 Trainable params: 2,369 (9.25 KB)

 Non-trainable params: 0 (0.00 B)

- 또는 다음과 같은 구문을 사용할 수 있습니다.

In [ ]:
# 메모리 정리
clear_session()

# 입력 값 개수
nfeatures = x.shape[1]

# Sequential 모델 선언
model = Sequential()
model.add(Input(shape=(nfeatures, )))
model.add(Dense(1))

# 모델 요약
model.summary()

#### 4) 모델 학습

- 최적화 방법과 손실 함수를 지정해 모델 학습을 준비합니다.

In [23]:
# x, y 분리
target = 'Sales'
x = scaled_df.drop(columns=target)
y = scaled_df.loc[:, target]

In [24]:
# 학습 설정
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

- 학습용 데이터로 모델 학습을 수행합니다.

In [25]:
# 모델 학습
history = model.fit(x, y, epochs=200, batch_size=2, verbose=1)

Epoch 1/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 10.8772 - mae: 3.1437
Epoch 2/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6360 - mae: 1.0744
Epoch 3/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9012 - mae: 0.7233
Epoch 4/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6651 - mae: 0.6286
Epoch 5/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9118 - mae: 0.6885
Epoch 6/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5951 - mae: 0.6162
Epoch 7/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5949 - mae: 0.6029
Epoch 8/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4061 - mae: 0.5047
Epoch 9/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4545 - mae: 0.5147
Epoch 10/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3727 - mae: 0.4498
Epoch 11/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3267 - mae: 0.4233
Epoch 12/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4628 - mae: 0.5207
Epoch 13/200

#### 5) 예측 및 성능 평가

- 검증용 데이터에 대해 예측을 수행합니다.

In [26]:
# 예측
y_pred = model.predict(x)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [45]:
# 실젯값, 예측값 확인
compare = pd.DataFrame({
    '실젯값(Sales)': y,
    '예측값(Predicted)': y_pred.flatten()
})
compare.head()

ValueError: array length 200 does not match index length 400

- 여러 평가 지표로 모델의 성능을 평가합니다.

In [31]:
# 성능 평가
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"모델 성능 평가")
print(f"---------------------------")
print(f"MSE(평균제곱오차): {mse:.4f}")
print(f"MAE(평균절대오차): {mae:.4f}")
print(f"R²(결정계수): {r2:.4f}")

모델 성능 평가
---------------------------
MSE(평균제곱오차): 0.0138
MAE(평균절대오차): 0.0799
R²(결정계수): 0.9862


## **3. 모델링 2: 카시트 판매량 에측**

### (1) 데이터 전처리

- 데이터 준비
- 가변수화
- 데이터 분할
- 스케일링

#### 1) 데이터 준비

- 경로: 'https://raw.githubusercontent.com/Jangrae/csv/master/carseats.csv'

In [32]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/carseats.csv'
data = pd.read_csv(path)
data.drop(columns='Education', inplace=True)
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Urban,US
0,9.50,138,73,11,276,120,Bad,42,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,Yes,No


**데이터 설명**

- Sales: 각 지역 판매량(단위: 1,000개)
- CompPrice: 경쟁사 가격(단위: 달러)
- Income: 지역 평균 소득(단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산(단위: 1,000달러)
- Population: 지역 인구수(단위: 1,000명)
- Price: 자사 지역별 판매가격
- ShelveLoc: 진열상태
- Age: 지역 인구의 평균 연령
- Urban: 도심 지역 여부(Yes, No)
- US: 매장이 미국에 있는지 여부(Yes, No)

In [33]:
# x, y 분리
target = 'Sales'
x = data.drop(columns=target)
y = data.loc[:, target]

#### 2) 가변수화

In [34]:
cat_cols = ['ShelveLoc', 'US', 'Urban']
x = pd.get_dummies(x, columns=cat_cols, drop_first=True, dtype=int)

#### 3) 데이터 분할

In [35]:
# 학습용, 검증용 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

#### 4) 스케일링

In [37]:
# 스케일링
num_cols = ['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age']

scaler = StandardScaler()
x_train_scaled = x_train.copy()
x_train_scaled[num_cols] = scaler.fit_transform(x_train[num_cols])

x_val_scaled = x_val.copy()
x_val_scaled[num_cols] = scaler.transform(x_val[num_cols])

### (2) 모델링

- 필요한 함수들 불러오기
- 모델 선언
- 학습
- 예측 및 성능 평가

In [39]:
# 함수 불러오기
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### 1) 모델 선언

In [40]:
# 메모리 정리
tf.keras.backend.clear_session()

# 입력 값 개수
nfeatures = x_train_scaled.shape[1]
print(nfeatures)

# Sequential 모델 선언
model = Sequential([
    Dense(64, activation='relu', input_shape=(nfeatures,)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# 모델 요약
model.summary()

10


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,817 (11.00 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 0 (0.00 B)

#### 2) 모델 학습

In [41]:
# 학습 설정
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [43]:
# 모델 학습
history = model.fit(x_train_scaled, y_train, epochs=200, batch_size=2, verbose=1)

Epoch 1/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 10.5437 - mae: 2.3817
Epoch 2/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.2577 - mae: 1.3784
Epoch 3/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6939 - mae: 1.2778
Epoch 4/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.5144 - mae: 1.2499
Epoch 5/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.2945 - mae: 1.1840
Epoch 6/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.4296 - mae: 1.2370
Epoch 7/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.4840 - mae: 1.2138
Epoch 8/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.9799 - mae: 1.1268
Epoch 9/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1167 - mae: 1.1520
Epoch 10/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.4329 - mae: 1.1940
Epoch 11/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2673 - mae: 1.1899
Epoch 12/200
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7980 - mae: 1.0723
Epoch 13/200

#### 3) 예측 및 성능 평가

In [48]:
# 예측
y_pred = model.predict(x_val_scaled)

# 실젯값, 예측값 확인
compare = pd.DataFrame({
    '실젯값(Sales)': y_val,
    '예측값(Predicted)': y_pred.flatten()
})
compare.head()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


,실젯값(Sales),예측값(Predicted)
398,5.94,5.307055
125,9.34,12.261280
328,3.15,4.665319
339,11.54,10.071020
172,9.03,11.143695


In [51]:
# 성능 평가
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"모델 성능 평가")
print(f"---------------------------")
print(f"MSE(평균제곱오차): {mse:.4f}")
print(f"MAE(평균절대오차): {mae:.4f}")
print(f"R²(결정계수): {r2:.4f}")

모델 성능 평가
---------------------------
MSE(평균제곱오차): 2.0654
MAE(평균절대오차): 1.1496
R²(결정계수): 0.7511


## **4. 모델링 3: 집값 예측**

### (1) 데이터 전처리

#### 1) 데이터 준비

In [62]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/boston.csv'
data = pd.read_csv(path)
data.drop(columns='black', inplace=True)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


**데이터 설명**

- crim: 자치시(Town)별 1인당 범죄율
- zn: 25,000 평방피트를 초과하는 거주지역 비율
- indus: 비소매상업지역이 점유하고 있는 토지 비율
- chas: 찰스강에 대한 더미 변수 (= 1 강 경계에 위치; 0 나머지)
- nox: 10ppm당 농축 일산화질소
- rm: 주택 1가구당 평균 방 개수
- age: 1940년 이전에 건축된 소유주택 비율
- dis: 5개 보스턴 직업센터까지 접근성 지수
- rad: 방사형 도로까지의 접근성 지수
- tax: 10,000달러 당 재산세율
- lstat: 모집단 하위 계층의 비율(%)
- medv: 본인 소유 주택가격(중앙값) (단위:$1,000)

In [63]:
# x, y 분리
target = 'medv'
x = data.drop(columns=target)
y = data.loc[:, target]

#### 2) 가변수화

In [64]:
# 없음

#### 3) 데이터 분할

In [65]:
# 학습용, 검증용 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

#### 4) 스케일링

In [ ]:
# 스케일링
num_cols = x.select_dtypes(include=['int64', 'float64']).columns

scaler = StandardScaler()
x_train_scaled = x_train.copy()
x_train_scaled[num_cols] = scaler.fit_transform(x_train[num_cols])

x_val_scaled = x_val.copy()
x_val_scaled[num_cols] = scaler.transform(x_val[num_cols]) 


### (2) 모델링

- 함수 불러오기
- 모델 선언
- 모델 학습
- 예측 및 성능 평가

In [67]:
# 함수 불러오기
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### 1) 모델 선언

In [68]:
# 메모리 정리
tf.keras.backend.clear_session()

# 입력 값 개수
nfeatures = x_train_scaled.shape[1]
print(nfeatures)

# Sequential 모델 선언
model = Sequential([
    Dense(64, activation='relu', input_shape=(nfeatures,)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# 모델 요약
model.summary()

12


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

#### 2) 모델 학습

In [69]:
# 학습 설정
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [70]:
# 모델 학습
history = model.fit(x_train_scaled, y_train, epochs=200, batch_size=2, verbose=1)

Epoch 1/200


202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 281.5765 - mse: 281.5765
Epoch 2/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.1208 - mse: 35.1208
Epoch 3/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.7756 - mse: 26.7756
Epoch 4/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 26.9141 - mse: 26.9141
Epoch 5/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.8956 - mse: 24.8956
Epoch 6/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.3274 - mse: 26.3274
Epoch 7/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.0814 - mse: 22.0814
Epoch 8/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 22.1985 - mse: 22.1985
Epoch 9/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 21.4321 - mse: 21.4321
Epoch 10/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 21.8074 - mse: 21.8074
Epoch 11/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 19.7711 - mse: 19.7711
Epoch 12/200
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 19.1645 - mse: 19.1645

#### 3) 예측 및 성능 평가

In [71]:
# 예측
y_pred = model.predict(x_val_scaled)

# 실젯값, 예측값 확인
compare = pd.DataFrame({
    '실젯값(medv)': y_val,
    '예측값(Predicted)': y_pred.flatten()
})
compare.head()

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/stepWARNING:tensorflow:5 out of the last 14 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000246788A6CA0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


,실젯값(Sales),예측값(Predicted)
307,28.2,29.200968
343,23.9,22.108459
47,16.6,19.926867
67,22.0,20.782946
362,20.8,23.968252


In [72]:
# 성능 평가
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"모델 성능 평가")
print(f"---------------------------")
print(f"MSE(평균제곱오차): {mse:.4f}")
print(f"MAE(평균절대오차): {mae:.4f}")
print(f"R²(결정계수): {r2:.4f}")

모델 성능 평가
---------------------------
MSE(평균제곱오차): 6.5567
MAE(평균절대오차): 1.9951
R²(결정계수): 0.9337
